# Generation of PMO Version 2.0
## 6 May 2024

In [98]:
import datetime
import pathlib
import urllib.parse
import xml.etree.ElementTree as etree

import rdflib

from jinja2 import Template

DC = rdflib.Namespace("http://purl.org/dc/terms/")
OWL = rdflib.Namespace("http://www.w3.org/2002/07/owl#")
RDACT = rdflib.Namespace("http://rdaregistry.info/termList/RDACarrierType/")
SKOS = rdflib.Namespace("http://www.w3.org/2004/02/skos/core#")

In [2]:
rdf_template = """<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF xmlns:dcterms="http://purl.org/dc/terms/"
         xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
         xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
         xmlns:skos="http://www.w3.org/2004/02/skos/core#"
         xmlns:rdact="http://rdaregistry.info/termList/RDACarrierType/">
</rdf:RDF>"""

ontology_html_template ="""<!doctype html>
<html lang="en">
<head>
  <link href="/css/owl.css" rel="stylesheet" type="text/css" />
  <link href="/css/Primer.css" rel="stylesheet" type="text/css" />
  <link href="/css/rec.css" rel="stylesheet" type="text/css" />
  <link href="/css/extra.css" rel="stylesheet" type="text/css" />
  <link rel="shortcut icon" href="/css/favicon.ico" />
</head>
<body>
    <h1>{label}</h1>
    <p>
      Documentation at <a href="/2.0/ontologies/vocabularies/{html_doc}">{label}</a>.
    </p>
    <ul>
    {entities_list}
    </ul>
</body>
</html>"""

html_template = """<!doctype html>
<html lang="en">
<head>
  <link href="/css/owl.css" rel="stylesheet" type="text/css" />
  <link href="/css/Primer.css" rel="stylesheet" type="text/css" />
  <link href="/css/rec.css" rel="stylesheet" type="text/css" />
  <link href="/css/extra.css" rel="stylesheet" type="text/css" />
  <link rel="shortcut icon" href="/css/favicon.ico" />
</head>
<body>
  <h1>{label}</h1>
  <p>
      Documentation at <a href='/2.0/ontologies/vocabularies/{html_doc}#{name}'>{label}</a>.
      Other versions:
      <ul>
          <li><a href="/2.0/ontologies/vocabularies/{}/{name}.rdf">RDF XML</a></li>
          <li><a href="/2.0/ontologies/vocabularies/{}/{name}.ttl">Turtle</a></li>
      </ul>
  </p>
</body>
</html>"""

In [145]:
main_html_template = Template("""<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<link href="/css/owl.css" rel="stylesheet" type="text/css">
<link href="/css/Primer.css" rel="stylesheet" type="text/css">
<link href="/css/rec.css" rel="stylesheet" type="text/css">
<link href="/css/extra.css" rel="stylesheet" type="text/css">
<link rel="shortcut icon" href="/PerformedMusicOntology/css/favicon.ico">
<script src="/js/jquery.js"></script>
<script src="/PerformedMusicOntology/js/jquery.scrollTo.js"></script>
<script src="/PerformedMusicOntology/js/marked.min.js"></script>
<title>{{ vocab.title }}</title>
</head>
<body>
<div class="head">
<h3>{{ vocab.title }}</h3>
<p>{{ vocab.url }}</p>
<div class="comment">
  <span class="markdown">
    <p>{{ vocab.definition }}</p>
  </span>
  <p><a href="{{ vocab.url }}.rdf">Vocabulary source</a></p>
</div>
<hr>
<div id="toc">
  <h2>Table of Contents</h2>
  <ol>
   <li><a href="#classes">Classes</a></li>
   <li><a href="#namespacedeclarations">Namespace Declarations</a></li>
  </ol>
</div>
<div id="classes">
  <h2>Classes</h2>
  <ul class="hlist">
  {% for class_ in classes %}
    <li>
      <a href="#{{ class_.id }}" title="{{ class_.url }}">
        <span>{{ class_.prefLabel }}</span>
      </a>
    </li>
  {% endfor %}
  </ul>
</div>
</div>
{% for class_ in classes %}
<div id="{{ class_.id }}" class="entity">
  <a name="{{ class_.url }}"></a>
  <h3>
   {{ class_.prefLabel }}<sup title="class" class="type-c">c</sup>
   <span class="backlink"> back to <a href="#toc">ToC</a> or <a href="#classes">Class ToC</a></span>
  </h3>
  <p><strong>IRI: </strong>{{ class_.url }}</p>
  {% if class_.altLabel %}
  <p><strong>Use for: </strong>{{ class_.altLabel }}</p>
  {% endif %}
  <p><strong>Scope note: </strong>{{ class_.definition }}</p>
  <p><strong>Current version: </strong>{{ class_.changeNote }}</p>
</div>
</div>
{% endfor %}
<div id="namespacedeclarations">
  <h2>Namespace Declarations <span class="backlink"> back to <a href="#toc">ToC</a></span></h2>
    <dl>
    <dt>dcterms</dt>
    <dd>http://purl.org/dc/terms/</dd>
    <dt>rdf</dt>
    <dd>http://www.w3.org/1999/02/22-rdf-syntax-ns#</dd>
    <dt>rdfs</dt>
    <dd>http://www.w3.org/2000/01/rdf-schema#</dd>
    <dt>skos</dt>
    <dd>http://www.w3.org/2004/02/skos/core#</dd>
    </dl>
</div>
</body>
</html>
""")

In [3]:
version_2_0 = pathlib.Path("/Users/jpnelson/30-39 LD4P, PCC, and Sinopia/35.01 LD4P Performed Music Ontology/2.0")

In [4]:
version_2_0.parent.mkdir?

Signature: version_2_0.parent.mkdir(mode=511, parents=False, exist_ok=False)
Docstring: Create a new directory at this given path.
File:      /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pathlib.py
Type:      method

In [156]:
def init_ontology(**kwargs):
    rdf_path = pathlib.Path(kwargs.get("rdf_path"))
    html_doc_file = kwargs.get("html_doc_file")
    start = datetime.datetime.now(datetime.UTC)
    print("{} Staring ontology initialization".format(start.isoformat()))
    if not rdf_path.exists():
        raise ValueError(f"{rdf_path} rdf_path doesn't exist")
    ontology_graph = rdflib.Graph()
    ontology_graph.parse(rdf_path, format='turtle')
    ontology_entity = ontology_graph.value(predicate=rdflib.RDF.type, object=SKOS.ConceptScheme)
    ontology_label = ontology_graph.value(subject=ontology_entity, predicate=rdflib.RDFS.label)
    entities = list(ontology_graph.subjects(predicate=rdflib.RDF.type, object=SKOS.Concept))
    ontology_uri = urllib.parse.urlparse(str(ontology_entity))
    ontology_path = pathlib.Path(f"../{ontology_uri.path}")
    print(ontology_path.absolute(), ontology_path.exists())
    if not ontology_path.exists():
        ontology_path.mkdir(parents=True, exist_ok=True)
    generate_vocab_home(
        ontology_entity,
        ontology_graph,
        ontology_path,
        html_doc_file,
        entities)
    entity_list_html = ''
    for i,row in enumerate(entities):
        if i > 0 and not i%10:
            print(f"{i:2}", end="")
        else:
            print(".", end="")
        entity_list_html += "{}\n".format(
            init_entity(entity=row,
                        ontology_name=ontology_label,
                        ontology_path=ontology_path,
                        ontology_graph=ontology_graph,
                        html_doc_file=html_doc_file)
            )
    raw_html = ontology_html_template.format(**{
            "label": ontology_label,
            "html_doc": html_doc_file,
            "entities_list": entity_list_html})
    with (ontology_path / "index.html").open("w+") as fo:
        fo.write(raw_html)
    end = datetime.datetime.now(datetime.UTC)
    print(f"\n{end.isoformat()} - Finished {ontology_label}, total time {(end-start).seconds / 60.0:,} minutes")

def generate_vocab_home(ontology_entity, graph, ontology_path, html_file_name, entities):
    vocab = {
        "title": graph.value(subject=ontology_entity, predicate=DC.title),
        "url": str(ontology_entity),
        "definition":  graph.value(subject=ontology_entity, predicate=SKOS.definition)
    }
    classes = []
    for entity in entities:
        class_ = {
            "id": str(entity).split("/")[-1],
            "definition": graph.value(subject=entity, predicate=SKOS.definition),
            "prefLabel":  graph.value(subject=entity, predicate=SKOS.prefLabel),
            "changeNote": _change_note(entity, graph),
            "url": str(class_uri)
        
        }
        alt_label = graph.value(subject=entity, predicate=SKOS.altLabel)
        if alt_label:
            class_["altLabel"] = alt_label
        classes.append(class_)
    with (ontology_path.parent / html_file_name).open("w+") as fo:
        fo.write(main_html_template.render(vocab=vocab, classes=classes))
        
def _change_note(uri, graph):
    change_note_bnode = graph.value(subject=uri, predicate=SKOS.changeNote)
    note_val = graph.value(subject=change_note_bnode, predicate=rdflib.RDF.value)
    date = graph.value(subject=change_note_bnode, predicate=DC.date)
    return f"{note_val} {date}"

def init_entity(**kwargs):
    entity = kwargs.get('entity')
    ontology_name = kwargs.get("ontology_name")
    ontology_path = kwargs.get("ontology_path")
    ontology_graph = kwargs.get("ontology_graph")
    html_doc_file = kwargs.get("html_doc_file")

    name = str(entity).split("/")[-1]

    # Make sub-directory if it doesn't exist
    entity_path = ontology_path / name
    if not entity_path.exists():
        entity_path.mkdir(parents=True, exist_ok=True)
    # Create an HTML file that redirects to Entity
    html_path = entity_path / "index.html"
    label = ontology_graph.value(subject=entity, predicate=SKOS.prefLabel)
    if not html_path.exists():
        with html_path.open("w+") as fo:
            fo.write(html_template.format(**{ "html_doc": html_doc_file,
                                              "label": label, 
                                              "name": name }))

    graph = rdflib.ConjunctiveGraph()
    graph.namespace_manager.bind("dc", DC)
    graph.namespace_manager.bind("rdact", RDACT)
    graph.namespace_manager.bind("skos", SKOS)
    for pred, obj in ontology_graph.predicate_objects(entity):
        graph.add((entity, pred, obj))
        if isinstance(obj, rdflib.BNode):
            for bnode_pred, bnode_obj in ontology_graph.predicate_objects(obj):
                graph.add((obj, bnode_pred, bnode_obj))
    # Writes an RDF XML file if it doesn't exist
    rdf_xml_path = entity_path / f"{name}.rdf"
    save_rdf(graph, rdf_xml_path, "xml")
    # Writes a RDF Turtle file
    rdf_ttl_path = entity_path / f"{name}.ttl"
    save_rdf(graph, rdf_ttl_path, "ttl") 
    print("\t{time} Finished initializing {label}".format(
          **{ "time": datetime.datetime.now(datetime.UTC).isoformat(),
            "label": label }))
    
    return f"""<li><a href="{entity}">{label}</li>"""
    
def save_rdf(graph, path, serialize_as):
    if not path.exists():
        print("\t\t{} Saving {}, {} triples as {}".format(datetime.datetime.now(datetime.UTC).isoformat(),
                                                  path,
                                                  len(graph),
                                                  serialize_as))
        #path.mkdir(parents=True, exist_ok=True)
        with path.open("w+") as fo:
            fo.write(graph.serialize(format=serialize_as))
    print(".", end="")

def generate_resource(subject: rdflib.URIRef):
    name = str(subject).split("/")[-1]
    resource_graph = rdflib.ConjunctiveGraph()
    resource_graph.namespace_manager.bind("dc", DC)
    resource_graph.namespace_manager.bind("rdact", RDACT)
    resource_graph.namespace_manager.bind("skos", SKOS)
    resource_path = ontology_path / name
    resource_path.mkdir(exist_ok=True)
    # label = pmo_2_0.value(subject=subject, predicate=rdflib.RDFS.label)
    # if label is None:
    #     label = name
    # definition = pmo_2_0.value(subject=subject, predicate=SKOS.definition)
    # raw_html = html_template.format(name=name, label=label, definition=definition)
    # index_path = resource_path/"index.html"
    # index_path.write_text(raw_html)
    # Populate resource graph
    # for pred, obj in pmo_2_0.predicate_objects(subject=subject):
    #     resource_graph.add((subject, pred, obj))
    # Write RDF XML and Turtle serialization
    # rdf_path = resource_path/f"{name}.rdf"
    # rdf_path.write_text(resource_graph.serialize(format='xml'))
    # ttl_path = resource_path/f"{name}.ttl"
    # ttl_path.write_text(resource_graph.serialize(format='turtle'))

In [81]:
version_2_0.absolute()

PosixPath('/Users/jpnelson/30-39 LD4P, PCC, and Sinopia/35.01 LD4P Performed Music Ontology/2.0')

## Ensemble Size

In [146]:
init_ontology(rdf_path = (version_2_0 / "Ontology/Vocabularies/EnsembleSize/Ensemble_size.ttl"),
              html_doc_file="PMOEnsembleSize.html")

2024-05-07T18:11:07.091742+00:00 Staring ontology initialization
/Users/jpnelson/30-39 LD4P, PCC, and Sinopia/35.01 LD4P Performed Music Ontology/Documentation/../2.0/vocabularies/ensemble_size False
.		2024-05-07T18:11:07.103859+00:00 Saving ../2.0/vocabularies/ensemble_size/solo_or_individual/solo_or_individual.rdf, 8 triples as xml
.		2024-05-07T18:11:07.104528+00:00 Saving ../2.0/vocabularies/ensemble_size/solo_or_individual/solo_or_individual.ttl, 8 triples as ttl
.	2024-05-07T18:11:07.105316+00:00 Finished initializing solo or individual
.		2024-05-07T18:11:07.106405+00:00 Saving ../2.0/vocabularies/ensemble_size/duo/duo.rdf, 9 triples as xml
.		2024-05-07T18:11:07.107138+00:00 Saving ../2.0/vocabularies/ensemble_size/duo/duo.ttl, 9 triples as ttl
.	2024-05-07T18:11:07.108146+00:00 Finished initializing duo
.		2024-05-07T18:11:07.109129+00:00 Saving ../2.0/vocabularies/ensemble_size/trio/trio.rdf, 8 triples as xml
.		2024-05-07T18:11:07.109714+00:00 Saving ../2.0/vocabularies/ens

## EventType

In [147]:
init_ontology(rdf_path = (version_2_0 / "Ontology/Vocabularies/EventType/Event_type.ttl"),
              html_doc_file="PMOEventType.html")

2024-05-07T18:11:09.397671+00:00 Staring ontology initialization
/Users/jpnelson/30-39 LD4P, PCC, and Sinopia/35.01 LD4P Performed Music Ontology/Documentation/../2.0/vocabularies/event_type False
.		2024-05-07T18:11:09.407961+00:00 Saving ../2.0/vocabularies/event_type/audition/audition.rdf, 8 triples as xml
.		2024-05-07T18:11:09.408539+00:00 Saving ../2.0/vocabularies/event_type/audition/audition.ttl, 8 triples as ttl
.	2024-05-07T18:11:09.409272+00:00 Finished initializing Audition
.		2024-05-07T18:11:09.410206+00:00 Saving ../2.0/vocabularies/event_type/benefit_concert/benefit_concert.rdf, 8 triples as xml
.		2024-05-07T18:11:09.410747+00:00 Saving ../2.0/vocabularies/event_type/benefit_concert/benefit_concert.ttl, 8 triples as ttl
.	2024-05-07T18:11:09.411406+00:00 Finished initializing Benefit concert
.		2024-05-07T18:11:09.412461+00:00 Saving ../2.0/vocabularies/event_type/ceremony/ceremony.rdf, 8 triples as xml
.		2024-05-07T18:11:09.412994+00:00 Saving ../2.0/vocabularies/eve

## Medium Component Qualifier

In [149]:
init_ontology(rdf_path = (version_2_0 / "Ontology/Vocabularies/MediumComponentQualifier/Medium_component_qualifier.ttl"),
              html_doc_file="PMOMediumComponentQualifier.html")

2024-05-07T18:20:49.685041+00:00 Staring ontology initialization
/Users/jpnelson/30-39 LD4P, PCC, and Sinopia/35.01 LD4P Performed Music Ontology/Documentation/../vocabularies/medium_component_qualifier False
.		2024-05-07T18:20:49.693062+00:00 Saving ../vocabularies/medium_component_qualifier/ad_lib/ad_lib.rdf, 9 triples as xml
.		2024-05-07T18:20:49.693650+00:00 Saving ../vocabularies/medium_component_qualifier/ad_lib/ad_lib.ttl, 9 triples as ttl
.	2024-05-07T18:20:49.694340+00:00 Finished initializing ad lib
.		2024-05-07T18:20:49.695525+00:00 Saving ../vocabularies/medium_component_qualifier/amplified/amplified.rdf, 8 triples as xml
.		2024-05-07T18:20:49.696063+00:00 Saving ../vocabularies/medium_component_qualifier/amplified/amplified.ttl, 8 triples as ttl
.	2024-05-07T18:20:49.696700+00:00 Finished initializing amplified
.		2024-05-07T18:20:49.697568+00:00 Saving ../vocabularies/medium_component_qualifier/obbligato/obbligato.rdf, 8 triples as xml
.		2024-05-07T18:20:49.698159+00

In [36]:
pmo_1_0 = rdflib.ConjunctiveGraph()
pmo_1_0.parse("/Users/jpnelson/30-39 LD4P, PCC, and Sinopia/35.01 LD4P Performed Music Ontology/ontologies/PerformedMusicOntology.rdf")
pmo_2_0 = rdflib.ConjunctiveGraph()
pmo_2_0.parse("/Users/jpnelson/30-39 LD4P, PCC, and Sinopia/35.01 LD4P Performed Music Ontology/2.0/Ontology/PMO.rdf")

<Graph identifier=N9120e3484cf84220a213a1398907b9b1 (<class 'rdflib.graph.Graph'>)>

In [46]:
for pred, obj in pmo_2_0.predicate_objects(subject=dramatic_role):
    print(f"Version 2.O: {pred},{obj}")

Version 2.O: http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#Class
Version 2.O: http://www.w3.org/2000/01/rdf-schema#label,Dramatic role
Version 2.O: http://www.w3.org/2004/02/skos/core#definition,A dramatic role performed by a contributing agent.
Version 2.O: http://www.w3.org/2004/02/skos/core#example,Role of "Carmen" in the opera Carmen.


In [157]:
for rdf_type in [OWL.Class, OWL.ObjectProperty, OWL.DatatypeProperty]:
    print(f"Adding {rdf_type}")
    for s in pmo_2_0.subjects(predicate=rdflib.RDF.type, object=rdf_type):
        generate_resource(s)
        print(str(s).split("/")[-1])

Adding http://www.w3.org/2002/07/owl#Class


NameError: name 'ontology_path' is not defined

In [101]:
ontology_graph = rdflib.Graph()
ontology_graph.parse((version_2_0 / "Ontology/Vocabularies/EventType/Event_type.ttl"), format='turtle')

<Graph identifier=N247485a5628a40ef81119565eb1dbcb8 (<class 'rdflib.graph.Graph'>)>

In [102]:
event_type_uri = rdflib.URIRef("http://performedmusicontology.org/2.0/vocabularies/event_type/")
vocab = {
    "title": ontology_graph.value(subject=event_type_uri, predicate=DC.title),
    "url": str(event_type_uri),
    "definition":  ontology_graph.value(subject=event_type_uri, predicate=SKOS.definition)
}

In [116]:
def _change_note(uri, graph):
    change_note_bnode = graph.value(subject=uri, predicate=SKOS.changeNote)
    note_val = graph.value(subject=change_note_bnode, predicate=rdflib.RDF.value)
    date = graph.value(subject=change_note_bnode, predicate=DC.date)
    return f"{note_val} {date}"

In [118]:
classes = []
for class_uri in ontology_graph.subjects(predicate=rdflib.RDF.type, object=SKOS.Concept):
    class_ = {
        "id": str(class_uri).split("/")[-1],
        "definition": ontology_graph.value(subject=class_uri, predicate=SKOS.definition),
        "prefLabel": ontology_graph.value(subject=class_uri, predicate=SKOS.prefLabel),
        "changeNote": _change_note(class_uri, ontology_graph),
        "url": str(class_uri)
        
    }
    classes.append(class_)

In [119]:
html = main_html_template.render(vocab=vocab, classes=classes)

In [150]:
def convert_ttl_rdf_xml(rdf_filepath, output_path):
    graph = rdflib.Graph()
    graph.parse(rdf_filepath, format='turtle')
    with output_path.open("w+") as fo:
        fo.write(graph.serialize(format='xml'))


PosixPath('/Users/jpnelson/30-39 LD4P, PCC, and Sinopia/35.01 LD4P Performed Music Ontology/2.0/Ontology/Vocabularies/EnsembleSize/Ensemble_size.ttl')

In [151]:
convert_ttl_rdf_xml(
    (version_2_0 / "Ontology/Vocabularies/EnsembleSize/Ensemble_size.ttl"),
    (version_2_0 / "vocabularies/ensemble_size/PMOEnsembleSize.rdf"))

In [152]:
convert_ttl_rdf_xml(
    (version_2_0 / "Ontology/Vocabularies/EventType/Event_type.ttl"),
    (version_2_0 / "vocabularies/event_type/PMOEventType.rdf"))

In [153]:
convert_ttl_rdf_xml(
    (version_2_0 / "Ontology/Vocabularies/MediumComponentQualifier/Medium_component_qualifier.ttl"),
    (version_2_0 / "vocabularies/medium_component_qualifier/PMOMediumComponentQualifier.rdf"))

## PMO Version 2.0 Page

In [ ]:
template = Template("""<?xml version="1.0" encoding="UTF-8"?>
<html xmlns="http://www.w3.org/1999/xhtml">
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <link href="/css/owl.css" rel="stylesheet" type="text/css" />
    <link href="/css/Primer.css" rel="stylesheet" type="text/css" />
    <link href="/css/rec.css" rel="stylesheet" type="text/css" />
    <link href="/css/extra.css" rel="stylesheet" type="text/css" />
    <link rel="shortcut icon" href="/css/favicon.ico" />
    <script src="/js/jquery.js"></script>
    <script src="/js/jquery.scrollTo.js"></script>
    <script src="/PerformedMusicOntology/js/marked.min.js"></script>
    <script>
      $(document).ready(
            function () {
                jQuery(".markdown").each(function(el){
                    jQuery(this).after(marked(jQuery(this).text())).remove()});
                var list = $('a[name="http://www.essepuntato.it/static/tmp/0"]');
                if (list.size() != 0) {
                    var element = list.first();
                    $.scrollTo(element);
                }
        });
    </script>
  </head>
  <body>
    <div class="head">
      <h1>Performed Music Ontology 2.0</h1>
      <dl>
        <dt>IRI:</dt>
        <dd>http://performedmusicontology.org/2.0/ontology/</dd>
        <dt>Version IRI:</dt>
        <dd>http://performedmusicontology.org/2.0/ontology/</dd>
       </dl>
       <dl>
         <dt>Other visualisation:</dt>
         <dd><a href="http://performedmusicontology.org/2.0/ontology/PerformedMusicOntology.rdf">Ontology source</a></dd>
        </dl>
     </div>
     <hr />
     <div id="toc">
       <h2>Table of Content</h2>
       <ol>
          <li><a href="#classes">Classes</a></li>
          <li><a href="#objectproperties">Object Properties</a></li>
          <li><a href="#dataproperties">Data Properties</a></li>
          <li><a href="#namespacedeclarations">Namespace Declarations</a></li>
        </ol>
     </div>
     <div id="classes">
        <h2>Classes</h2>
        <ul class="hlist">
          {% for class_ in classes %}
            <li>
              <a href="#{{ class_.id }}" title="{{ class_.url }}">
                <span>{{ class_.prefLabel }}</span>
              </a>
            </li>
          {% endfor %}
        </ul>
        {% for class_ in classes %}
        <div id="{{ class_.id }}" class="entity">
          <a name="{{ class_.url }}"></a>
          <h3>
           {{ class_.prefLabel }}<sup title="class" class="type-c">c</sup>
           <span class="backlink"> back to <a href="#toc">ToC</a> or <a href="#classes">Class ToC</a></span>
          </h3>
          <p><strong>IRI: </strong>{{ class_.url }}</p>
          {% if class_.altLabel %}
          <p><strong>Use for: </strong>{{ class_.altLabel }}</p>
          {% endif %}
          <p><strong>Scope note: </strong>{{ class_.definition }}</p>
          <p><strong>Current version: </strong>{{ class_.changeNote }}</p>
        </div>
        {% endfor %}
      </div>
   </body>
</html>""")

In [164]:
for s in pmo_2_0.subjects():
    print(s)

http://performedmusicontology.org/ontology/phonogramCopyrightDate
http://performedmusicontology.org/ontology/hasMediumOfPerformance
http://performedmusicontology.org/ontology/Tempo
http://performedmusicontology.org/ontology/hasInspiration
http://performedmusicontology.org/ontology/hasPerformance
http://performedmusicontology.org/ontology/hasMediumCount
http://performedmusicontology.org/ontology/OpusNumber
http://performedmusicontology.org/ontology/hasMode
http://performedmusicontology.org/ontology/DramaticRole
http://performedmusicontology.org/ontology/RismNumber
http://performedmusicontology.org/ontology/createdFor
http://performedmusicontology.org/ontology/createdFor
http://performedmusicontology.org/ontology/ThematicCatalogNumber
http://performedmusicontology.org/ontology/performanceOf
http://performedmusicontology.org/ontology/DramaticRole
http://performedmusicontology.org/ontology/hasMode
http://performedmusicontology.org/ontology/hasDramaticRole
http://performedmusicontology.org/

In [163]:
for s in pmo_1_0.subjects():
    print(s)

http://performedmusicontology.org/ontology/ThematicCatalogStatement
http://performedmusicontology.org/ontology/MasterClass
http://performedmusicontology.org/ontology/hasTempo
http://performedmusicontology.org/ontology/hasMode
http://performedmusicontology.org/ontology/hasMediumOfPerformance
http://performedmusicontology.org/ontology/createdFor
http://performedmusicontology.org/ontology/Tempo
http://performedmusicontology.org/ontology/Tempo
http://performedmusicontology.org/ontology/hasNumberOfHands
http://performedmusicontology.org/ontology/asMemberOf
http://performedmusicontology.org/ontology/aggregates
http://performedmusicontology.org/ontology/hasMediumPartType
http://performedmusicontology.org/ontology/hasEnsembleCount
http://performedmusicontology.org/ontology/hasKeyMode
http://performedmusicontology.org/ontology/DramaticRole
http://performedmusicontology.org/ontology/DramaticRole
http://performedmusicontology.org/ontology/hasPerformance
http://performedmusicontology.org/ontology/